In [128]:
%matplotlib nbagg
import pandas as pd
import os, sys
import numpy as np
from collections import defaultdict
from caseDetection import detect_case
#import matplotlib as mpl
#mpl.use("GTK3cairo")
import matplotlib.pyplot as plt
#plt.plot([1,2,3,4],'*-')

# DATA ANALYSIS
- per country num obs
- per country num unique sIPs tested

#### reset index, as index were repeated due to concatenation earlier
#### this will not be a problem with any new dataset

In [60]:
df_all = pd.read_pickle("data/detected_case_all_Apr12.pkl").reset_index()

In [61]:
df_sIP_subcat = pd.read_csv("data/Servers_IMC.txt").set_index('sIP')
#df_sIP_subcat

In [162]:
SAMPLENAME = 'sample1'
RESULTS = "results/" + SAMPLENAME + "/"
if not os.path.exists(RESULTS):
    os.makedirs(RESULTS)

### SANITIZE (UNBIAS)
- get only countries with more than 400 unique sIPs
- select only one measurement for each unique sIPs in all countries

#### UNBIASED: so choose only good measurements...

In [62]:
df_good = df_all [  df_all['case'].isin([1,2,3])  ]

In [63]:
# meaningful num of countries with at least 400 unique SIPs
THRESHOLD = 400

# per country, how many unique sIPs
df_unique_sIP_per_country = df_good.groupby(['country'])['sIP'].unique().apply(lambda x:len(x))
df_current = df_unique_sIP_per_country[df_unique_sIP_per_country > THRESHOLD]

In [64]:
valid_countries = df_current.index
TOTAL_NUM_VALID_COUNTRIES = len(valid_countries)
print valid_countries, TOTAL_NUM_VALID_COUNTRIES

# filter df_good to only valid countries
# remove extra columns
df_valid = df_good[  df_good['country'].isin(valid_countries)   ] [['sIP', 'gIP', 'country', 'diff_list', 'domain', 'subcat', 'case']]

Index([u'BR', u'CA', u'CN', u'CR', u'CZ', u'DE', u'DK', u'ES', u'FR', u'GB', u'IN', u'IT', u'JP', u'KR', u'MX', u'MY', u'NL', u'NO', u'PH', u'PL', u'RU', u'SE', u'UA', u'US'], dtype='object') 24


### Unique sIPs per country
- use good cases only (no case 0, -1, 4)
- select only sIPs which measure to all countries TOTAL_NUM_VALID_COUNTRIES
- get their subcats
- get only one measurement per subcat

In [65]:
df_num_unique_countries_per_sIP = df_valid.groupby('sIP')['country'].unique().apply(lambda x: len(x))
df_num_unique_countries_per_sIP.sort(inplace=True)

# SELECT ONLY sIPs WHICH MEASURE TO ALL COUNTRIES
df_sIPs_all_countries = pd.DataFrame(df_num_unique_countries_per_sIP[df_num_unique_countries_per_sIP==TOTAL_NUM_VALID_COUNTRIES])
df_sIP_subcat_represent = df_sIPs_all_countries.join(df_sIP_subcat)

# THE UNBIASED MEASUREMENT
df_sIP_subcat_represent.groupby('subcat')['country'].count()

subcat
adult             16
arts              11
business          11
circum             7
computers          7
games              6
health            11
home               6
kids_and_teens     6
news               4
recreation        10
reference         14
regional           8
science           14
shopping           8
society           13
sports             8
vpn                9
world             11
Name: country, dtype: int64

In [66]:
good_sIPs = df_sIP_subcat_represent.index
print "Unbiased sIPs represented in all good measurements to all countries: ", len(good_sIPs)

Unbiased sIPs represented in all good measurements to all countries:  180


<h3>This data is unbiased because we remove bad cases and keep equal representation of sIPs for every country</h3>
- for these sIPs take a random measurement from each sIP, country group in df_good now

In [67]:
df_good_sIP = df_valid[df_valid['sIP'].isin(good_sIPs)]
gp_good_sIP_country = df_good_sIP.groupby(['sIP','country'])

# even though there are multiple cases per sIP, country pair
# select only one
df_per_sIP_country = gp_good_sIP_country['case'].unique().reset_index()
df_per_sIP_country['count'] = gp_good_sIP_country['case'].count().reset_index()['case']
df_per_sIP_country.head()

,sIP,country,case,count
0,1.209.122.23,BR,"[2, 1]",20
1,1.209.122.23,CA,[2],8
2,1.209.122.23,CN,[2],28
3,1.209.122.23,CR,[2],1
4,1.209.122.23,CZ,[2],2


#### choose one random index of each sip,country pair

In [70]:
# for every sIP, country pair, select a random index of measurement
useful_indices = defaultdict(int)
sanity_check = defaultdict(int)
    
for sIP_country, indices in gp_good_sIP_country.groups.items():
    useful_indices[sIP_country] = np.random.choice( indices )
    sanity_check[sIP_country]+=1
    
print len(useful_indices)
print {k:v for k,v in sanity_check.items() if v>1}

4320
{}


In [71]:
test_indices = useful_indices.values()[:30]

# UNBIASED DATA
- filter df_good_sIP to only the random indices selected
- code under this can be run on current measurements as every sIP, country has just 1 measurement

In [72]:
df_unbiased = df_good_sIP.ix[ useful_indices.values() ]
df_unbiased.to_pickle(RESULTS+"sampled_unbiased_CenP.pkl")
df_unbiased.head()

,sIP,gIP,country,diff_list,domain,subcat,case
14531,74.115.2.240,61.155.176.10,CN,"[3, 2, 1, 4, 1, None, None, 1, 0, 1, 3, 0, 2, ...",hsselite.com,vpn,3
77018,23.59.189.89,80.71.68.80,DK,"[5, 2, 2, 4, 6, 2, 2, 3, 2, 4, 2, 2, 3, 6, 4, ...",news.com.au,news,2
119742,74.117.180.88,78.156.128.125,CZ,"[8, 0, 1, 30, 17, 4, 0, 0, 2, 3, 28, 2, 6, 28,...",videosexarchive.com,adult,2
206627,74.115.2.240,130.206.211.70,ES,"[None, 5, 5, None, None, 4, 3, 3, 1, 3, 1, 2, ...",hsselite.com,vpn,2
145595,208.94.153.100,203.200.163.130,IN,"[None, 1, 1, 0, 0, 1, 0, 0, 1, 3, 0, 0, 0, 0, ...",southwest.com,recreation,2


In [79]:
# each country, sIP has just one entry in df_unbiased
len(df_unbiased)

4320

In [78]:
df_temp = df_unbiased.groupby(['sIP', 'country'])['case'].apply(lambda x: len(x))
df_temp[df_temp > 1]

Series([], name: case, dtype: int64)

In [80]:
grouped_by_country_subcat_case = df_unbiased.groupby(['country', 'subcat', 'case'])
count_per_country_subcat_case = grouped_by_country_subcat_case.count()
count_per_country_subcat_case.head(10)

sIP  gIP  diff_list  domain
country subcat    case                             
BR      adult     1       1    1          1       1
                  2      12   12         12      12
                  3       3    3          3       3
        arts      2      10   10         10      10
                  3       1    1          1       1
        business  1       1    1          1       1
                  2      10   10         10      10
        circum    2       7    7          7       7
        computers 2       5    5          5       5
                  3       2    2          2       2

In [81]:
# replace nan with 0
df_case_count = count_per_country_subcat_case.unstack(level=-1)['sIP'].fillna(0)
df_case_count['total'] = df_case_count.sum(axis=1)
df_case_count.to_html(RESULTS+'cases_per_country_subcat-unbiased-CenP.htm')
df_case_count.head()

case               1   2  3  total
country subcat                    
BR      adult      1  12  3     16
        arts       0  10  1     11
        business   1  10  0     11
        circum     0   7  0      7
        computers  0   5  2      7

#### Censorship by country for each subcat
- create num(subcat) bar plots comparing case2 with (case1 + case3)

In [83]:
# clear all except df_case_count
#del df, df_bad, df_case_default, df_current, df_good, df_merged_unique_cases, df_obs_per_country, df_redo
#del df_sIP_CO, df_temp, df_unique, df_unique_sIP_per_country

In [84]:
group_by_subcat = df_case_count.reset_index().groupby('subcat')

### Per Subcategory Analysis, Excluding error cases

In [85]:
def get_ratios(subcat, indices, df_case_count):
    #if subcat == 'all':
        
    temp_df = df_case_count.ix[ group_by_subcat.groups[subcat] ].reset_index()
    temp_df['censored'] = temp_df[1]+temp_df[3]
    #temp_df['unknown'] = temp_df[-1]+temp_df[0]+temp_df[4]
    #temp_df = temp_df.rename(columns={2:'uncensored'})[['country', 'censored', 'uncensored', 'unknown']].set_index('country')
    temp_df = temp_df.rename(columns={2:'uncensored'})[['country', 'censored', 'uncensored']].set_index('country')

    # calculate ratios
    temp_df['total'] = temp_df.sum(axis=1)
    temp_df['ratio-censored'] = temp_df['censored']/(temp_df['total'])
    temp_df['ratio-uncensored'] = temp_df['uncensored']/(temp_df['total'])
    #temp_df['ratio-unknown'] = temp_df['unknown']/(temp_df['total'])

    # replace no entries with 0
    #df_country_case = temp_df.fillna(0)
    return temp_df

In [86]:
country_case_per_subcat = defaultdict(int)

# iterate over groupby object, indices
for subcat, indices in group_by_subcat.groups.iteritems():
    country_case_per_subcat[subcat] = get_ratios(subcat, indices, df_case_count)
    print subcat, len(country_case_per_subcat[subcat])
    
df_country_case_subcat = pd.concat(country_case_per_subcat)

science 24
arts 24
regional 24
shopping 24
reference 24
business 24
world 24
kids_and_teens 24
computers 24
recreation 24
sports 24
society 24
games 24
adult 24
home 24
health 24
news 24
vpn 24
circum 24


In [163]:
df_country_case_subcat.head(10)

case           censored  uncensored  total  ratio-censored  ratio-uncensored
      country                                                               
adult BR              4          12     16          0.2500            0.7500
      CA              5          11     16          0.3125            0.6875
      CN              3          13     16          0.1875            0.8125
      CR              3          13     16          0.1875            0.8125
      CZ              5          11     16          0.3125            0.6875
      DE              5          11     16          0.3125            0.6875
      DK              4          12     16          0.2500            0.7500
      ES              3          13     16          0.1875            0.8125
      FR              5          11     16          0.3125            0.6875
      GB              5          11     16          0.3125            0.6875

In [88]:
all_results = list(df_country_case_subcat.columns)
print all_results

['censored', 'uncensored', 'total', 'ratio-censored', 'ratio-uncensored']


In [89]:
censorship_ratio = defaultdict(int)
for key in all_results:
    censorship_ratio[key] = df_country_case_subcat[key].unstack(0)
    censorship_ratio[key].to_html(RESULTS+'censorship_'+key+'.html')

In [90]:
censorship_ratio['ratio-censored']

,adult,arts,business,circum,computers,games,health,home,kids_and_teens,news,recreation,reference,regional,science,shopping,society,sports,vpn,world
country,,,,,,,,,,,,,,,,,,,
BR,0.2500,0.090909,0.090909,0.000000,0.285714,0.500000,0.363636,0.000000,0.000000,0.25,0.2,0.000000,0.125,0.142857,0.000,0.307692,0.125,0.222222,0.090909
CA,0.3125,0.090909,0.090909,0.000000,0.285714,0.500000,0.272727,0.333333,0.000000,0.50,0.0,0.000000,0.250,0.214286,0.125,0.230769,0.125,0.444444,0.090909
CN,0.1875,0.090909,0.090909,0.000000,0.571429,0.333333,0.454545,0.166667,0.166667,0.00,0.1,0.071429,0.250,0.142857,0.250,0.230769,0.000,0.555556,0.181818
CR,0.1875,0.090909,0.000000,0.000000,0.285714,0.500000,0.272727,0.166667,0.000000,0.00,0.0,0.000000,0.125,0.142857,0.000,0.307692,0.000,0.222222,0.090909
CZ,0.3125,0.090909,0.090909,0.285714,0.285714,0.500000,0.272727,0.333333,0.000000,0.00,0.1,0.000000,0.125,0.142857,0.125,0.307692,0.000,0.333333,0.181818
DE,0.3125,0.181818,0.181818,0.000000,0.285714,0.500000,0.181818,0.000000,0.166667,0.00,0.4,0.071429,0.125,0.142857,0.250,0.307692,0.125,0.444444,0.090909
DK,0.2500,0.090909,0.000000,0.000000,0.285714,0.500000,0.181818,0.000000,0.000000,0.00,0.0,0.000000,0.000,0.142857,0.000,0.307692,0.000,0.444444,0.090909
ES,0.1875,0.090909,0.000000,0.000000,0.285714,0.500000,0.272727,0.000000,0.166667,0.00,0.1,0.142857,0.125,0.142857,0.125,0.307692,0.000,0.333333,0.090909
FR,0.3125,0.090909,0.000000,0.142857,0.285714,0.500000,0.181818,0.000000,0.000000,0.00,0.0,0.000000,0.125,0.142857,0.000,0.384615,0.000,0.444444,0.090909


## PLOT CENSORSHIP

#### games category

In [117]:
df = df_unbiased[ df_unbiased['subcat']=='games' ]
game_censorship = pd.DataFrame( df.groupby(['case', 'domain'])['country'].unique() )
game_censorship['count'] = game_censorship['country'].apply( lambda x: len(x) )
game_censorship['country'] = game_censorship['country'].apply( lambda x: np.sort(x) )

In [142]:
game_censorship

country  \
case domain                                                                   
2    battle.net           [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     mmo-champion.com                                                  [CN]   
     pcgamer.com          [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     williamhill.com      [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
3    curse.com            [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     mmo-champion.com     [BR, CA, CR, CZ, DE, DK, ES, FR, GB, IN, IT, J...   
     planetminecraft.com  [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   

                          count  
case domain                      
2    battle.net              24  
     mmo-champion.com         1  
     pcgamer.com             24  
     williamhill.com         24  
3    curse.com               24  
     mmo-champion.com        23  
     planetminecraft.com     24

In [ ]:
game_censorship.to_html(RESULTS+subcat+'_censorship.html')

### Every subcat


In [159]:
print df_unbiased['subcat'].unique()

['vpn' 'news' 'adult' 'recreation' 'reference' 'computers' 'world' 'health'
 'arts' 'business' 'circum' 'sports' 'science' 'shopping' 'kids_and_teens'
 'home' 'games' 'regional' 'society']


In [160]:
for subcat in df_unbiased['subcat'].unique():
    df = df_unbiased[ df_unbiased['subcat'] == subcat ]
    censorship = pd.DataFrame( df.groupby(['case', 'domain'])['country'].unique() )
    censorship['count'] = censorship['country'].apply( lambda x: len(x) )
    censorship['country'] = censorship['country'].apply( lambda x: np.sort(x) )
    censorship.to_html(RESULTS + 'subcat_censorship_'+subcat+'.html')

In [161]:
censorship

country  \
case domain                                                                 
1    legacy.com                                              [CA, IT, RU]   
     slate.com                           [FR, NL, NO, PH, PL, SE, UA, US]   
2    collegeboard.org   [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     dailykos.com       [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     digg.com           [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     hollywoodlife.com                               [CA, IN, NL, PH, US]   
     legacy.com         [BR, CN, CR, CZ, DE, DK, ES, FR, GB, IN, JP, K...   
     menshealth.com     [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     nih.gov            [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     realsimple.com                  [CA, CN, IN, JP, KR, MX, MY, PH, US]   
     salon.com          [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     slate.com          [BR, CA, CN, CR, CZ, DE, DK, ES, GB, IN, IT, J...   
     snopes.com         [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
3    change.org         [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     dailykos.com                                                    [US]   
     digg.com                                                        [US]   
     goodsearch.com     [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     hollywoodlife.com  [BR, CN, CR, CZ, DE, DK, ES, FR, GB, IT, JP, K...   
     nih.gov                                                         [NO]   
     realsimple.com     [BR, CR, CZ, DE, DK, ES, FR, GB, IT, NL, NO, P...   

                        count  
case domain                    
1    legacy.com             3  
     slate.com              8  
2    collegeboard.org      24  
     dailykos.com          23  
     digg.com              23  
     hollywoodlife.com      5  
     legacy.com            21  
     menshealth.com        24  
     nih.gov               23  
     realsimple.com         9  
     salon.com             24  
     slate.com             16  
     snopes.com            24  
3    change.org            24  
     dailykos.com           1  
     digg.com               1  
     goodsearch.com        24  
     hollywoodlife.com     19  
     nih.gov                1  
     realsimple.com        15

# WORLD MAP